In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [24]:
g = dataset[0]

In [38]:
len(g.edata["__orig__"])

10556

In [39]:
len(g.ndata["label"])

2708

In [81]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [92]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']

    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute precision
        pred = logits.argmax(1)

        # Compute loss
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # Compute accuracy on trian/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, train_acc:{:.3f}, val_acc:{:.3f} (best val_acc:{:.3f}, test_acc:{:.3f} (best test_val:{:.3f})'.format(e, train_acc, val_acc, best_val_acc, test_acc, best_test_acc))
        
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)


    

/Users/coldeye/miniforge3/envs/torch/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, train_acc:0.129, val_acc:0.152 (best val_acc:0.152, test_acc:0.133 (best test_val:0.133)
In epoch 5, train_acc:0.571, val_acc:0.344 (best val_acc:0.396, test_acc:0.364 (best test_val:0.408)
In epoch 10, train_acc:0.771, val_acc:0.494 (best val_acc:0.494, test_acc:0.513 (best test_val:0.513)
In epoch 15, train_acc:0.864, val_acc:0.588 (best val_acc:0.588, test_acc:0.600 (best test_val:0.600)
In epoch 20, train_acc:0.914, val_acc:0.624 (best val_acc:0.624, test_acc:0.628 (best test_val:0.628)
In epoch 25, train_acc:0.950, val_acc:0.648 (best val_acc:0.650, test_acc:0.666 (best test_val:0.659)
In epoch 30, train_acc:0.957, val_acc:0.674 (best val_acc:0.674, test_acc:0.701 (best test_val:0.701)
In epoch 35, train_acc:0.964, val_acc:0.692 (best val_acc:0.692, test_acc:0.719 (best test_val:0.719)
In epoch 40, train_acc:0.964, val_acc:0.716 (best val_acc:0.716, test_acc:0.738 (best test_val:0.738)
In epoch 45, train_acc:0.971, val_acc:0.732 (best val_acc:0.732, test_acc:0.745 (bes

In [94]:
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 5, 4, 3]))

In [96]:
g.edges()

(tensor([0, 0, 0, 0, 0]), tensor([1, 2, 5, 4, 3]))

In [117]:
import dgl.function as fn

class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        self.linear = nn.Linear(2 * in_feat, out_feat)
    
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim = 1)
            return self.linear(h_total)
    
    

In [118]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feat, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feat)
        self.conv2 = SAGEConv(h_feat, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [120]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, model):
    all_logits = []
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']

    for e in range(100):
        logits = model(g, features)
        pred = logits.argmax(1)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean() 
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean() 
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean() 

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())
        
        if e % 5 == 0:
            print('In epoch {}, loss:{:.3f}, trian_acc:{:.3f}, val_acc:{:.3f} (best_val_acc:{:.3f}), test_acc:{:.3f} (best_test_acc:{:.3f})'.format(e, loss, train_acc, val_acc, best_val_acc, test_acc, best_test_acc))
    
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss:1.950, trian_acc:0.143, val_acc:0.122 (best_val_acc:0.122), test_acc:0.130 (best_test_acc:0.130)
In epoch 5, loss:1.896, trian_acc:0.714, val_acc:0.476 (best_val_acc:0.476), test_acc:0.450 (best_test_acc:0.450)


/Users/coldeye/miniforge3/envs/torch/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 10, loss:1.791, trian_acc:0.714, val_acc:0.256 (best_val_acc:0.476), test_acc:0.256 (best_test_acc:0.450)
In epoch 15, loss:1.635, trian_acc:0.814, val_acc:0.298 (best_val_acc:0.476), test_acc:0.287 (best_test_acc:0.450)
In epoch 20, loss:1.430, trian_acc:0.886, val_acc:0.356 (best_val_acc:0.476), test_acc:0.347 (best_test_acc:0.450)
In epoch 25, loss:1.186, trian_acc:0.957, val_acc:0.478 (best_val_acc:0.478), test_acc:0.447 (best_test_acc:0.447)
In epoch 30, loss:0.924, trian_acc:0.993, val_acc:0.610 (best_val_acc:0.610), test_acc:0.601 (best_test_acc:0.601)
In epoch 35, loss:0.674, trian_acc:1.000, val_acc:0.678 (best_val_acc:0.678), test_acc:0.671 (best_test_acc:0.671)
In epoch 40, loss:0.463, trian_acc:1.000, val_acc:0.698 (best_val_acc:0.700), test_acc:0.685 (best_test_acc:0.685)
In epoch 45, loss:0.304, trian_acc:1.000, val_acc:0.700 (best_val_acc:0.700), test_acc:0.701 (best_test_acc:0.685)
In epoch 50, loss:0.195, trian_acc:1.000, val_acc:0.720 (best_val_acc:0.720), te

In [107]:
logits = torch.tensor([[0, 1, 2, 3, 4], [1, 2, 3, 4, 5]])
logits.argmax(1k)

tensor([4, 4])

In [129]:
class FALayer(nn.Module):
    def __init__(self, g, in_dim, dropout):
        super(FALayer, self).__init__()
        self.g = g
        self.dropout = nn.Dropout(dropout)
        self.gate = nn.Linear(2 * in_dim, 1)
        nn.init.xavier_normal_(self.gate.weight, gain=1.414)

    def edge_applying(self, edges):
        h2 = torch.cat([edges.dst['h'], edges.src['h']], dim=1)
        g = torch.tanh(self.gate(h2)).squeeze()
        e = g * edges.dst['d'] * edges.src['d']
        e = self.dropout(e)
        return {'e': e, 'm': g}

    def forward(self, h):
        self.g.ndata['h'] = h
        self.g.apply_edges(self.edge_applying)
        self.g.update_all(fn.u_mul_e('h', 'e', '_'), fn.sum('_', 'z'))

        return self.g.ndata['z']

In [130]:
model = FALayer(g, g.ndata['feat'].shape[1], 0)

In [160]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [163]:
def edge_attention(edges):
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = attn_fc(z2)
        return {'e': F.leaky_relu(a)}

def message_func(edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

def reduce_func(nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        alpha = alpha * self.norm(self.g, self.g.edata['e'])
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

g.ndata['z'] = g.ndata['feat']
g.apply_edges(edge_attention)

NameError: name 'self' is not defined

In [330]:
from dgl.nn import EdgeWeightNorm, GATv2Conv
import numpy as np

class GPR_prop_attention(nn.Module):
    def __init__(self, g, K=10, alpha=0.1, Init='PPR', hidden_dim=1433, attention_dim=100, Gamma=None):
        super(GPR_prop_attention, self).__init__()
        self.K = K
        self.Init = Init
        self.alpha = alpha
        self.norm = EdgeWeightNorm(norm='both')

        assert Init in ['SGC', 'PPR', 'NPPR', 'Random', 'WS']
        if Init == 'SGC':
            # SGC-like, note that in this case, alpha has to be a integer. It means where the peak at when initializing GPR weights.
            TEMP = 0.0*np.ones(K+1)
            TEMP[alpha] = 1.0
        elif Init == 'PPR':
            # PPR-like
            TEMP = alpha*(1-alpha)**np.arange(K+1)
            TEMP[-1] = (1-alpha)**K
        elif Init == 'NPPR':
            # Negative PPR
            TEMP = (alpha)**np.arange(K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'Random':
            # Random
            bound = np.sqrt(3/(K+1))
            TEMP = np.random.uniform(-bound, bound, K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'WS':
            # Specify Gamma
            TEMP = Gamma
        
        self.temp = nn.Parameter(torch.tensor(TEMP))

        self.g = g
        self.fc = nn.Linear(hidden_dim, attention_dim, bias=False)
        self.attn_fc = nn.Linear(2 * attention_dim, 1, bias=False)

    
    def reset_parameters(self):
        torch.nn.init.zeros_(self.temp)
        for k in range(self.K+1):
            self.temp.data[k] = self.alpha*(1-self.alpha)**k
        self.temp.data[-1] = (1-self.alpha)**self.K
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.fc.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def edge_attention(self, edges):
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        a = a * self.norm(self.g, torch.ones(self.g.edata['e'].shape).squeeze()).unsqueeze(dim=1) 
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        return {'h': edges.src['h'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['h'], dim=1)
        return {'h': h}

    def forward(self, in_feat):
        self.g.ndata['h'] = in_feat
        self.g.ndata['TH'] = in_feat * (self.temp[0])
        for k in range(self.K):
            z = self.fc(self.g.ndata['h'])
            self.g.ndata['z'] = z
            self.g.apply_edges(self.edge_attention)
            self.g.update_all(self.message_func, self.reduce_func)
            gamma = self.temp[k+1]
            self.g.ndata['TH'] = self.g.ndata['TH'] + self.g.ndata['h'] * gamma
        return self.g.ndata['TH']

In [331]:
prop1 = GPR_prop_attention(g)

In [333]:
prop1(g.ndata['feat']).shape

torch.Size([2708, 1433])

In [301]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.fc.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        print(a.shape)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # print(nodes.mailbox['e'].shape)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

In [302]:
gat = GATLayer(g, g.ndata['feat'].shape[1], 100)

In [303]:
gat(g.ndata['feat'])

torch.Size([10556, 1])


tensor([[-4.1790e-03,  2.9552e-03, -5.6929e-03,  ...,  4.0570e-03,
         -7.2795e-04, -2.7860e-03],
        [-3.5868e-03, -1.6618e-03, -6.5353e-03,  ...,  3.9866e-04,
         -9.7184e-04, -3.5037e-03],
        [-3.5868e-03, -1.6618e-03, -6.5353e-03,  ...,  3.9866e-04,
         -9.7184e-04, -3.5037e-03],
        ...,
        [ 3.5966e-03,  1.7319e-03, -4.0267e-03,  ...,  2.1680e-02,
         -7.7565e-03, -7.4146e-03],
        [-2.1122e-06,  1.0050e-02, -1.4291e-02,  ...,  1.1471e-02,
          9.6459e-04,  1.6365e-02],
        [-5.2093e-03,  2.4313e-02, -2.4839e-03,  ...,  5.3332e-03,
         -1.6284e-02,  7.7768e-03]], grad_fn=<IndexCopyBackward0>)

In [304]:
torch.ones()

TypeError: ones() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [284]:
g.edata['e'].shape

torch.Size([10556, 1])

In [11]:
import dgl
import torch
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [15]:
len

tensor([0, 1, 2, 3, 4, 5, 6])

In [2]:
import networkx as nx
import numpy as np

dataset = 'cornell'
graph_adjacency_list_file_path = '../high_freq/{}/out1_graph_edges.txt'.format(dataset)
graph_node_features_and_labels_file_path = '../high_freq/{}/out1_node_feature_label.txt'.format(dataset)

G = nx.DiGraph()
graph_node_features_dict = {}
graph_labels_dict = {}

with open(graph_node_features_and_labels_file_path) as graph_node_features_and_labels_file:
    graph_node_features_and_labels_file.readline()
    for line in graph_node_features_and_labels_file:
        line = line.rstrip().split('\t')
        assert (len(line) == 3)
        assert (int(line[0]) not in graph_node_features_dict and int(line[0]) not in graph_labels_dict)
        graph_node_features_dict[int(line[0])] = np.array(line[1].split(','), dtype=np.uint8)
        graph_labels_dict[int(line[0])] = int(line[2])

with open(graph_adjacency_list_file_path) as graph_adjacency_list_file:
    graph_adjacency_list_file.readline()
    for line in graph_adjacency_list_file:
        line = line.rstrip().split('\t')
        assert (len(line) == 2)
        if int(line[0]) not in G:
            G.add_node(int(line[0]), features=graph_node_features_dict[int(line[0])],
                       label=graph_labels_dict[int(line[0])])
        if int(line[1]) not in G:
            G.add_node(int(line[1]), features=graph_node_features_dict[int(line[1])],
                       label=graph_labels_dict[int(line[1])])
        G.add_edge(int(line[0]), int(line[1]))

 

In [8]:
sorted(G.nodes(data='features'))

[(0, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (1, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (2, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (3, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (4, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (5, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (6, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (7, array([1, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (8, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (9, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (10, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (11, array([0, 0, 0, ..., 1, 0, 0], dtype=uint8)),
 (12, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (13, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (14, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (15, array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)),
 (16, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (17, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (18, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)),
 (19, array([0, 0, 0, 

NameError: name 'g' is not defined

In [37]:
u = torch.tensor([0, 1, 2, 3, 4, 5])
v = torch. tensor([1, 2, 3, 4, 5, 0])
g = dgl.graph((u, v))
train_ = np.array([0, 1, 2])
val = np.array([3, 4])
test = np.array([5])
n_nodes = 6
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[train_] = True
val_mask[val] = True
test_mask[test] = True
g.ndata['train_mask'] = train_mask
g.ndata['train_mask']


tensor([ True,  True,  True, False, False, False])

In [43]:
n = 927
train_rate = 0.3
x = int(n * train_rate)
x = np.ones((20, 1))
len(x)

20

In [23]:
y = int(n * (1 - train_rate) / 2)
y

324

In [4]:
import os
os.path.abspath("../result")

'/Users/coldeye/Documents/2_Current_Year/FYP/code/GPR-GNN_Attention/result'

In [5]:
a = "ss" + "sb"
a

'sssb'

In [6]:
"haha" + "_record"+ ".txt"

'haha_record.txt'

In [18]:
import torch
import dgl
from dataset_utils import preprocess_data
from dgl.data import CoraGraphDataset

In [3]:
dataset = preprocess_data('squirrel', 0.6) 
dataset

WikipediaNetwork()

In [ ]:
dataset = preprocess_data('chameleon', 0.6) 

In [20]:
g = dataset[0]
src = g.edge_index[0]
dst = g.edge_index[1]
graph = dgl.graph((src,dst), num_nodes=g.x.size(0))
graph.ndata['feat'] = g.x
graph.ndata['label'] = g.y

In [16]:
graph

Graph(num_nodes=5201, num_edges=217073,
      ndata_schemes={'feat': Scheme(shape=(2089,), dtype=torch.float32)}
      edata_schemes={})

In [22]:
graph1 = CoraGraphDataset()[0]
graph1.ndata['label'].unique().size(0)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


7

In [40]:
train_rate = 0.6
val_rate = 0.2
percls_trn = int(round(train_rate*len(graph.ndata['label'])/graph.ndata['label'].unique().size(0)))
val_lb = int(round(val_rate*len(graph.ndata['label'])))
TrueLBrate = (percls_trn*dataset.num_classes+val_lb)/len(graph.ndata['label'])
print('True Label rate: ', TrueLBrate)

True Label rate:  0.6285081240768094


In [81]:
graph = CoraGraphDataset(verbose=False)[0]

In [41]:
percls_trn

232

In [42]:
indices = []
for i in range(7):
        index = (graph.ndata['label'] == i).nonzero().view(-1)
        index = index[torch.randperm(index.size(0))]
        indices.append(index)

In [48]:
graph.ndata['label'][indices[1][2]]

tensor(1)

In [51]:
dataset == dgl.data.AmazonCoBuyPhotoDataset()
graph = dataset[0]

Extracting file to /Users/coldeye/.dgl/amazon_co_buy_photo


In [71]:
from torch_geometric.datasets import WebKB, WikipediaNetwork
import torch_geometric.transforms as T
dataset = WikipediaNetwork(root='../data/',
                        name='chameleon', transform=T.NormalizeFeatures(),  geom_gcn_preprocess=False)
g = dataset[0]
src = g.edge_index[0]
dst = g.edge_index[1]
graph = dgl.graph((src,dst), num_nodes=g.x.size(0))
graph.ndata['feat'] = g.x
graph.ndata['label'] = g.y

Data(x=[2277, 128], edge_index=[2, 62792], y=[2277])

In [70]:
graph.ndata['label']

tensor([ 5.1475,  8.9984,  9.0559,  ...,  4.6052, 10.0388,  5.6240])

In [79]:
import numpy as np
a = np.array([1,2, 3])
b = np.array([3,4, 5])
c = [a ,b]

In [80]:
np.var(c, axis=0)

array([1., 1., 1.])

In [ ]:
g